# Chart Understanding Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/minimal_data_example.ipynb)

In [127]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [128]:
import os
from dotenv import load_dotenv
import pandas as pd
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp
from urllib.parse import urlparse

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [129]:
# inputs

url = 'https://app.netdata.cloud/spaces/netdata-demo/rooms/all-nodes/overview#metrics_correlation=false&after=1680156258000&before=1680171603000&utc=Europe%2FLondon&offset=%2B1&timezoneName=Edinburgh%2C%20London&modal=&modalTab=&d8a4e0c5-7c79-4145-900e-83a9f06fcb6a-overview-chartName=menu_apps_submenu_cpu&modalParams=&highlight_after=1680167698000&highlight_before=1680170347000'
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'd8a4e0c5-7c79-4145-900e-83a9f06fcb6a'
context = 'apps.cpu'
dimensions = ['containers', 'httpd', 'kube-proxy', 'sql']
freq = '3m'

url_parsed = urlparse(url)
url_fragments = {x.split('=')[0]:x.split('=')[1] for x in url_parsed.fragment.split('&')}
highlight_after = int(int(url_fragments.get('highlight_after')) / 1000)
highlight_before = int(int(url_fragments.get('highlight_before') )/ 1000)
after = int(int(url_fragments.get('after')) / 1000)
before = int(int(url_fragments.get('before')) / 1000)

print(space_id)
print(room_id)
print(context)
print(after) 
print(before)
print(highlight_after) 
print(highlight_before)

ea93d7b8-0df6-45c0-b13d-1560996c89eb
d8a4e0c5-7c79-4145-900e-83a9f06fcb6a
apps.cpu
1680156258
1680171603
1680167698
1680170347


In [130]:
df = get_data_cloud(space_id, room_id, context, after, before, freq=freq, dimensions=dimensions).add_prefix(f'{context}.')
df_highlight = get_data_cloud(space_id, room_id, context, highlight_after, highlight_before, freq=freq, dimensions=dimensions).add_prefix(f'{context}.')

df_time_min = df.index.min()
df_time_max = df.index.max()
df_highlight_time_min = df_highlight.index.min()
df_highlight_time_max = df_highlight.index.max()

# save tokens by using t
df['t'] = range(len(df))
df = df.set_index('t')
# try save tokens by using ints
df = (df*100).round(0).ffill().bfill().astype('int')
print(df.shape)
print(df.head())

# save tokens by using t
df_highlight['t'] = range(len(df_highlight))
df_highlight = df_highlight.set_index('t')
# try save tokens by using ints
df_highlight = (df_highlight*100).round(0).ffill().bfill().astype('int')
print(df_highlight.shape)
print(df_highlight.head())

(84, 4)
   apps.cpu.containers  apps.cpu.httpd  apps.cpu.kube-proxy  apps.cpu.sql
t                                                                        
0                 1079            1122                   59         16641
1                 1088            1746                   58         16863
2                 1081            1537                   61         16517
3                 1090            1441                   59         15884
4                 1082            1370                   60         16032
(14, 4)
   apps.cpu.containers  apps.cpu.httpd  apps.cpu.kube-proxy  apps.cpu.sql
t                                                                        
0                 1078            1158                   63         16410
1                 1087            1665                   63         15982
2                 1077            1826                   58         16350
3                 1077            1370                   59         15911
4                 1121

In [131]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring a server using Netdata Cloud and would like to compare some a highlighted window of data with a baseline.

Below is the some {context} metrics for this server from {df_time_min} to {df_time_max}, aggregated to a {freq} frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does it look like anything has changed? If so, what do you think has changed? Are there any potential issues we might need to investigate further?

Baseline metrics data:

{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}

Highlighted window metrics data:

{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}

"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring a server using Netdata Cloud and would like to compare some a highlighted window of data with a baseline.

Below is the some apps.cpu metrics for this server from 2023-03-30 06:09:33 to 2023-03-30 10:19:56, aggregated to a 3m frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does it look like anything has changed? If so, what do you think has changed? Are there any potential issues we might need to investigate further?

Baseline metrics data:

    apps.cpu.containers  apps.cpu.httpd  apps.cpu.kube-proxy  apps.cpu.sql
t                                                                         
0                  1079            1122                   59         16641
1                  1088            1746                   58         16863
2                  1081            1537                   61         16517
3                  1090            1441       

In [132]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [133]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [134]:
reply_content = completion.choices[0].message.content
pp.pprint(reply_content)
#print(reply_content)

('Based on the highlighted window, it does not seem like anything has changed '
 'significantly compared to the baseline metrics data. All the metrics are '
 'within a reasonable range and do not show any abnormal spikes or drops.\n'
 '\n'
 "However, it's worth noting that apps.cpu.sql seems to have decreased near "
 'the end of the highlighted window, potentially indicating an issue with the '
 'SQL application. This might warrant further investigation, such as looking '
 'into any recent changes or updates to the SQL application or database.')
